In [18]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal
import pandas as pd

# **Partie 2**

## **Fonctions generiques**

In [19]:
signal_dict = np.load('apprentissage/a_1.npz') # signal_dict est un dictionnaire
#contenant les données contenues dans le fichier signal.npz
s = signal_dict['s'] # extraction du signal utile
t = signal_dict['t'] # extraction du vecteur temps associé au signal

px.line(x=t, y=s, labels={"x":"Temps (s)", "y":"Amplitude"},
title="a_1.npz").show()

#help(san.trans_fourier)
nu, S = san.trans_fourier(s-np.mean(s), t)


px.line(x=nu, y=np.log10(np.absolute(S)), labels={"x":"Temps (s)", "y":"Amplitude"},
title="TF a_1.npz").show()

## **analyse des groupes sonores**

lettre = "a"
EnsembleVect = []
vect_freq_moyen = [0]
vect_ampl_moyen = [0]

courbes = []

for i in range (1,6):
    nom = str(lettre) +"_" +str(i) + ".npz"
    signal_dict = np.load('apprentissage/'+nom) 
    
    s = signal_dict['s'] # extraction du signal utile
    t = signal_dict['t'] # extraction du vecteur temps associé au signal
    
    """px.line(x=t, y=s, labels={"x":"Temps (s)", "y":"Amplitude"},
    title=nom).show()"""

    nu, S = san.trans_fourier(s-np.mean(s), t)
    S = np.absolute(S)
    S = S / np.max(S)  #normalisation de l'amplitude
    fondamentale = (san.detect_fondamentale(S, nu))
    S = np.log10(S) +2
    courbes.append(S)

    
    freqs_harmoniques = []
    for l in range (6):
        freqs_harmoniques.append((l+1)*fondamentale)
    #A partir d'ici un fait directement la moyenne des fondamentales et harmoniques
    vect_freq_moyen[0] += (1/5)*fondamentale #terminer la liste de toutes les freqs moyennes et amplitudes
    
    print('san detect pics',san.detect_pics(S,freqs_harmoniques,nu))
    valtest = san.detect_pics(S,freqs_harmoniques,nu) #tuple (freq pic, amplitude pic)

    harmoniques_reelles = valtest[0] #freqeunces reeles des harmoniques
    amplitudes_harmoniques = valtest[1] #amplitudes des harmoniques
    
    print("harmoniques",harmoniques_reelles)

    print("liste freqences pics",vect_freq_moyen)
    print("liste amplitudes pics",vect_ampl_moyen)

df_signaux = pd.DataFrame({
"Temps (Hz)": np.concatenate([nu, nu, nu, nu, nu]),
"Valeur": np.concatenate([courbes[0], courbes[1], courbes[2], courbes[3], courbes[4]]),
"Signal": ["S1"]*len(nu) + ["S2"]*len(nu) + ["S3"]*len(nu) + ["S4"]*len(nu) + ["S5"]*len(nu)
})
px.line(df_signaux, x="Temps (Hz)", y="Valeur", color="Signal"
,title="superposition des courbes").show()

In [48]:
LETTRES = ["a","e","i","o","u"]
dico = {"a":[0,0],"e":[0,0],"i":[0,0],"o":[0,0],"u":[0,0]}



for lettre in LETTRES:
    courbes = []
    for i in range (1,6):
        nom = str(lettre) +"_" +str(i) + ".npz"
        signal_dict = np.load('apprentissage/'+nom) 
        
        s = signal_dict['s'] # extraction du signal utile
        t = signal_dict['t'] # extraction du vecteur temps associé au signal
        
        """px.line(x=t, y=s, labels={"x":"Temps (s)", "y":"Amplitude"},
        title=nom).show()"""

        nu, S = san.trans_fourier(s-np.mean(s), t)
        S = np.absolute(S)
        fondamentale = (san.detect_fondamentale(S, nu, 0.7*np.max(S)))
        S = np.log10(S)
        S = S / np.max(S)  #normalisation de l'amplitude
        print(fondamentale)
        courbes.append(S)
        

        
        freqs_harmoniques = [fondamentale]
        for l in range (6): #ajout des 6 harmoniques suivantes
            freqs_harmoniques.append((l+1)*fondamentale)
        #A partir d'ici un fait directement la moyenne des fondamentales et harmoniques
        
        Lettre_Harmonique_Freq, Lettre_Harmonique_Amp= san.detect_pics(S,freqs_harmoniques,nu) #tuple (freq pic, amplitude pic)
        #Variables importantes ^
        
    dico[lettre][0] = Lettre_Harmonique_Freq
    dico[lettre][1] = Lettre_Harmonique_Amp
    print(lettre)
    df_signaux = pd.DataFrame({
    "Temps (Hz)": np.concatenate([nu, nu, nu, nu, nu]),
    "Valeur": np.concatenate([courbes[0], courbes[1], courbes[2], courbes[3], courbes[4]]),
    "Signal": ["S1"]*len(nu) + ["S2"]*len(nu) + ["S3"]*len(nu) + ["S4"]*len(nu) + ["S5"]*len(nu)
    })
    px.line(df_signaux, x="Temps (Hz)", y="Valeur", color="Signal"
    ,title=("superposition des courbes "+str(lettre))).show()


106.96433630555556
113.96200316666666
114.9616698611111
110.96300308333333
116.96100325
a


129.9566702777778
118.96033663888889
141.95267061111113
111.96266977777778
95.96800266666668
e


137.95400383333333
126.95767019444445
138.95367052777777
139.95333722222222
139.95333722222222
i


122.95900341666666
120.95967002777779
110.96300308333333
133.95533705555556
131.95600366666667
o


143.952004
140.95300391666666
133.95533705555556
141.95267061111113
143.952004
u
